In [ ]:
# | default_exp gs_curve

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# | export
import numpy as np
#from sureau_ecos_py.plc_comp import plc_comp

In [ ]:
# | export
def gs_curve(x:float, # Unknown parameter definition
             slope_gs:float, # Unknown parameter definition
             p50_gs:float, # Unknown parameter definition
             psi_start_closing:float, # Unknown parameter definition
             psi_close:float, # Unknown parameter definition
             pi_ft:float, # Osmotic potential at full turgor (MPa)
             e_symp:float, # Modulus of elastoicoty of the Symplasm (MPa/%)
             turgor_pressure_at_gs_max:float, # Unknown parameter definition
             gs_max:float, # Unknown parameter definition
             transpiration_model: str = ["jarvis", "granier"], # Transpiration model type
             stomatal_reg_formulation: str = [
                 "sigmoid",
                 "piecewise_linear",
                 "turgor",
                 ],  # type of regulation to be used for stomatal response to leaf symplasmic water potential, either `sigmoid` or `piecewise_linear`
             ) -> float:

    "To obtain plots of the gs regulation curve"

    assert stomatal_reg_formulation in [
        "sigmoid",
        "piecewise_linear",
        "turgor",
    ], f'{stomatal_reg_formulation} not a valid option, choose "sigmoid", "piecewise_linear" or "turgor" '

    assert transpiration_model in [
        "jarvis",
        "granier",
    ], f'{transpiration_model} not a valid option, choose  "jarvis" or "granier"'


    if transpiration_model == "granier":
        gs_max = 1

    if stomatal_reg_formulation == "sigmoid":
        pl_gs = 1 / (1 + np.exp(slope_gs / 25 * (x - p50_gs)))
        regul_fact = (1 - pl_gs)

    if stomatal_reg_formulation == "piecewise_linear":
        regul_fact = (x - psi_close) / (psi_start_closing - psi_close)

        if regul_fact < 0:
            regul_fact = 0

        elif regul_fact > 1:
            regul_fact = 1

        else:
            print("Original code don't take into account regul_fact between 0 and 1")

    if stomatal_reg_formulation == "turgor":
        # Only Rs1 is needed above TLP
        rs1 = (-1 * (x + pi_ft - e_symp) - np.sqrt((x + pi_ft - e_symp)**2 + 4 * (x * e_symp))) / (2 * e_symp)

        # turgor loss point variable created but not used
        turgor_loss_point = (pi_ft * e_symp)/(pi_ft + e_symp)

        turgor = - pi_ft - e_symp * rs1
        regul_fact <- turgor/turgor_pressure_at_gs_max

        if regul_fact < 0:
            regul_fact = 0

        elif regul_fact > 1:
            regul_fact = 1

        else:
            print("Original code don't take into account regul_fact between 0 and 1")

    return regul_fact*gs_max